# Import packages

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
!which python; python -V;

/usr/bin/python
Python 2.7.17


# Load dataset

In [3]:
movies_dataframe = pd.read_csv('../dataset/movies_with_features.csv', index_col='Id', encoding='utf-8')
movies_dataframe.head(10)
movies_dataframe = movies_dataframe.head(4000)

In [4]:
def getRatingsPathFromMovieID(id):
    basepath = '../dataset/training_set/mv_'
    fileEnding = '.txt'
    if(id<10):
        return basepath + '000000' + str(id) + fileEnding
    elif(id>=10 and id<100):
        return basepath + '00000' + str(id) + fileEnding
    elif(id>=100 and id<1000):
        return basepath + '0000' + str(id) + fileEnding
    elif(id>=1000 and id<10000):
        return basepath + '000' + str(id) + fileEnding
    elif(id>=10000 and id<100000):
        return basepath + '00' + str(id) + fileEnding
    else:
        return -1

In [5]:
movie_rankings = list()
for i in range(1,len(movies_dataframe.index)+1):
    path = getRatingsPathFromMovieID(i)
    try:
        movie_rankings.append(pd.read_csv(path, skiprows=[0], header=None))
        movie_rankings[i-1].columns = ['CustomerID', 'Rating', 'Date']
        movie_rankings[i-1]['MovieID'] = i
        #print('Read rankings for MovieID: ',str(i),'/',str(len(movies_dataframe.index)))
    except:
        print('Could not read txt file: ', path)

In [6]:
movie_rankings[6]

,CustomerID,Rating,Date,MovieID
0,951709,2,2001-11-04,7
1,585247,1,2003-12-19,7
2,2625420,2,2004-06-03,7
3,2322468,3,2003-11-12,7
4,2056324,2,2002-11-10,7
...,...,...,...,...
88,1712385,2,2005-09-14,7
89,1666693,1,2001-09-08,7
90,1494385,4,2002-02-03,7
91,2297901,2,2003-10-28,7


In [7]:
#Remove movies with fewer than 100 rankings
print(movies_dataframe.shape)
for i in range(0,len(movie_rankings)):
    if(len(movie_rankings[i])<100):
        movies_dataframe = movies_dataframe.drop(i)
print(movies_dataframe.shape)

(4000, 7)
(3775, 7)


In [8]:
movies_dataframe.head(10)

,MovieID,YearOfRelease,Title,Budget,Genre,Original_language,Runtime
Id,,,,,,,
0,1,2003.0,Dinosaur Planet,0,NaN,NaN,0.0
1,2,2004.0,Isle of Man TT 2004 Review,0,NaN,NaN,0.0
2,3,1997.0,Character,4500000,History,nl,122.0
3,4,1994.0,Paula Abdul's Get Up & Dance,0,NaN,NaN,0.0
4,5,2004.0,The Rise and Fall of ECW,0,NaN,NaN,0.0
5,6,1997.0,Sick,0,NaN,NaN,0.0
7,8,2004.0,What the #$*! Do We Know!?,0,Documentary,en,109.0
9,10,2001.0,Fighter,0,NaN,NaN,0.0
10,11,1999.0,Full Frame: Documentary Shorts,0,NaN,NaN,0.0


In [9]:
customer_movies = pd.DataFrame(columns=['CustomerID', 'MovieID', 'Rating', 'Date'])
customer_movies = pd.concat(movie_rankings)
customer_movies.head()

,CustomerID,Rating,Date,MovieID
0,1488844,3,2005-09-06,1
1,822109,5,2005-05-13,1
2,885013,4,2005-10-19,1
3,30878,4,2005-12-26,1
4,823519,3,2004-05-03,1


In [10]:
#Remove users with fewer than 15 ratings as we do not have a foundation to build our recoomenadations on.
print(customer_movies.shape)
customer_movies = customer_movies.groupby('CustomerID').filter(lambda x : len(x)>=15)
print(customer_movies.shape)

(21409524, 4)
(20208325, 4)


In [11]:
movies_with_customer_ratings_and_all_features = pd.merge(customer_movies, movies_dataframe, on="MovieID")

In [12]:
#Populate empty fields and NaN values
movies_with_customer_ratings_and_all_features = movies_with_customer_ratings_and_all_features.drop(['Budget','Genre','Original_language','Runtime'], axis=1)

In [18]:
#Convert Date to unix-timestamp
import datetime as dt
movies_with_customer_ratings_and_all_features['Date'] = pd.to_datetime(movies_with_customer_ratings_and_all_features['Date'])
movies_with_customer_ratings_and_all_features['Date'] = (movies_with_customer_ratings_and_all_features['Date'] - dt.datetime(1970,1,1)).dt.total_seconds()

In [ ]:
movies_with_customer_ratings_and_all_features.head()

In [ ]:
movies_with_customer_ratings_and_all_features.to_csv('../dataset/prepared/movies_with_customer_ratings_and_all_features.csv', encoding='utf-8')


https://blog.echen.me/2011/10/24/winning-the-netflix-prize-a-summary/

http://www.cs.carleton.edu/cs_comps/0607/recommend/recommender/results.html